# Tech Challenge

## Descrição

Você foi contratado(a) para uma consultoria e seu trabalho envolve analisar os dados de vitivinicultura da Embrapa, os quais estão disponíveis [aqui](http://vitibrasil.cnpuv.embrapa.br/index.php).

A ideia do projeto é a criação de uma _API_ pública de consulta nos dados do site nas respectivas abas:

* Produção
* Processamento
* Comercialização
* Importação
* Exportação

A _API_ vai servir para alimentar uma base de dados que futuramente será usada para um modelo de _Machine Learning_.

Seus objetivos incluem:

* Criar uma Rest _API_ em [Python](https://www.python.org/) que faça a consulta no [site da Embrapa](http://vitibrasil.cnpuv.embrapa.br/index.php).
* Sua _API_ deve estar documentada.
* É recomendável (não obrigatório) a escolha de um método de autenticação ([`JWT`](https://jwt.io), por exemplo).
* Criar um plano para fazer o _deploy_ da _API_, desenhando a arquitetura do projeto desde a ingestão até a alimentação do modelo (aqui não é necessário elaborar um modelo de ML, mas é preciso que vocês escolham um cenário interessante em que a _API_ possa ser utilizada).
* Fazer um _MVP_ realizando o _deploy_ com um link compartilhável e um repositório no [GitHub](https://github.com/LucasVmigotto/fiap-pos-ml-eng-techchallenge-1).

## Setup

### Importação de Libs

In [2]:
from os.path import exists
from os import listdir
from urllib.request import urlretrieve
from pathlib import Path

import numpy as np
import pandas as pd
import seaborn
from tqdm import tqdm
from matplotlib import pyplot as plt

### Constantes

In [3]:
BASE_URL = 'http://vitibrasil.cnpuv.embrapa.br/download/'

CSV_URLS = dict(
    producao='Producao.csv',
    processamento=dict(viniferas='ProcessaViniferas.csv',
                       americanas='ProcessaAmericanas.csv',
                       mesa='ProcessaMesa.csv',
                       sem_classificacao='ProcessaSemclass.csv'),
    comercializacao='Comercio.csv',
    importacao=dict(mesa='ImpVinhos.csv',
                    espumantes='ImpEspumantes.csv',
                    frescas='ImpFrescas.csv',
                    passas='ImpPassas.csv',
                    suco='ImpSuco.csv'),
    exportacao=dict(mesa='ExpVinho.csv',
                    espumantes='ExpEspumantes.csv',
                    frescas='ExpUva.csv',
                    suco='ExpSuco.csv'),
)

In [47]:
PATH_BASE_DATA = Path('./data')
PATH_RAW_DATA = Path('./data') / 'raw'
PATH_INTERIM_DATA = Path('./data') / 'interim'

SEED = 42

np.random.seed(SEED)

PATH_RAW_DATA.mkdir(parents=True, exist_ok=True)
PATH_INTERIM_DATA.mkdir(parents=True, exist_ok=True)

### Utils

In [39]:
def download_files(url: str, save_path: Path, is_retry: bool = False) -> None:
    try:
        if not exists(save_path):
            urlretrieve(url, save_path)
    except ConnectionResetError:
        print(f'File {url} could not be downloaded')
        if is_retry:
            download_files(url, save_path, True)
        return

In [40]:
def download_data_files(data_files_structure: dict, parent_path: Path = Path('')):
    for key, value in tqdm(data_files_structure.items()):
        if isinstance(value, dict):
            download_data_files(value, parent_path / key)
            continue
        (PATH_RAW_DATA / parent_path).mkdir(parents=True,
                                            exist_ok=True)
        download_files(url=BASE_URL + value,
                       save_path=PATH_RAW_DATA / parent_path / value.lower())

## Coleta dos Dados

In [41]:
download_data_files(CSV_URLS)
listdir(PATH_RAW_DATA)

100%|██████████| 5/5 [00:05<00:00,  1.18s/it]


['comercio.csv',
 'importacao',
 'data.zip',
 'producao.csv',
 'exportacao',
 '.gitkeep',
 'estimativa_dou_2020.xls',
 'processamento']

* Verificar o header de cada arquivo `.csv` - conferir o tipo de separador utilizado

In [64]:
! head -n 1 ./data/raw/**/*.csv

==> ./data/raw/exportacao/expespumantes.csv <==
Id;País;1970;1970;1971;1971;1972;1972;1973;1973;1974;1974;1975;1975;1976;1976;1977;1977;1978;1978;1979;1979;1980;1980;1981;1981;1982;1982;1983;1983;1984;1984;1985;1985;1986;1986;1987;1987;1988;1988;1989;1989;1990;1990;1991;1991;1992;1992;1993;1993;1994;1994;1995;1995;1996;1996;1997;1997;1998;1998;1999;1999;2000;2000;2001;2001;2002;2002;2003;2003;2004;2004;2005;2005;2006;2006;2007;2007;2008;2008;2009;2009;2010;2010;2011;2011;2012;2012;2013;2013;2014;2014;2015;2015;2016;2016;2017;2017;2018;2018;2019;2019;2020;2020;2021;2021;2022;2022

==> ./data/raw/exportacao/expsuco.csv <==
Id;País;1970;1970;1971;1971;1972;1972;1973;1973;1974;1974;1975;1975;1976;1976;1977;1977;1978;1978;1979;1979;1980;1980;1981;1981;1982;1982;1983;1983;1984;1984;1985;1985;1986;1986;1987;1987;1988;1988;1989;1989;1990;1990;1991;1991;1992;1992;1993;1993;1994;1994;1995;1995;1996;1996;1997;1997;1998;1998;1999;1999;2000;2000;2001;2001;2002;2002;2003;2003;2004;2004;2005;2005;200

**Um pouco de cultura**

* "Imaginemos, por um instante, que a humanidade fosse transportada a um país utópico, onde os pombos voem já assados, onde todo o alimento cresça do solo espontaneamente, onde cada homem encontre sua amada ideal e a conquiste sem qualquer dificuldade. Ora, nesse país, muitos homens morreriam de tédio ou se enforcariam nos galhos das árvores, enquanto outros se dedicariam a lutar entre si e a se estrangular, a se assassinar uns aos outros." (Arthur Schopenhauer)

**TL;DR**

* Ciência de dados desbanca Schopenhauer

**Whatever**

* Todos, com exceção de `processamento` que usa `\t`, usam, como separador, o `;`

## Manipulação dos Dados

## Exploração dos Dados